In [4]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=f78b8225b4f5bed07e7e75deaeec3c617f3a7e02af610f7db5f0227ffa339a05
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [30]:
import cv2
import numpy as np
from deepface import DeepFace
import numpy as np
from scipy.spatial.distance import cosine, euclidean # Import necessary distance metrics
import pickle
import os
from pathlib import Path
from deepface import DeepFace
import argparse
from google.colab.patches import cv2_imshow

In [25]:
!git clone https://github.com/AliNajafpour/AI-Face-Identifier.git

Cloning into 'AI-Face-Identifier'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 48 (delta 12), reused 11 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 297.10 KiB | 18.57 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
def embedding(image, model='ArcFace'):
    result = DeepFace.represent(img_path=image, model_name=model, enforce_detection=False)
    embedding = result[0]["embedding"]
    return np.array(embedding)

In [3]:
def preprocess_image(image_path, size=(160, 160)):
    img = cv2.imread(image_path)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    enhanced_bgr = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

    bright = cv2.convertScaleAbs(enhanced_bgr, alpha=1.1, beta=10)

    result = cv2.resize(bright, size, interpolation=cv2.INTER_LINEAR)

    return result

In [22]:
def cosine_similarity(emb1, emb2):
    if emb1 is None or emb2 is None:
        return -1.0
    # Normalize embeddings to unit length
    emb1 = emb1 / np.linalg.norm(emb1)
    emb2 = emb2 / np.linalg.norm(emb2)
    return np.dot(emb1, emb2)

def find_matching_face(new_image_path, embeddings_dir, threshold=0.5):
    # گرفتن اطلاعات برای عکس جدیدی'
    new_embedding = embedding(new_image_path, model='ArcFace')
    if new_embedding is None:
        return {
            "match_found": False,
            "message": f"Could not extract embedding from {new_image_path}",
            "closest_match": None,
            "similarity": -1.0
        }
    similarity = cosine_similarity(new_embedding, embeddings_dir)
    # مقایسه با حداقل دقتی که در نظر گرفتیم
    if similarity >= threshold:
        return {
            "match_found": True,
            "similarity": similarity
        }
    else:
        return {
            "match_found": False,
            "similarity": similarity
        }


In [41]:
em1 = embedding('/content/AI-Face-Identifier/face test image/download.png')

In [47]:
find_matching_face('/content/AI-Face-Identifier/face test image/images.png', em1) # almost the same image

{'match_found': True, 'similarity': np.float64(0.9033914203655482)}

In [48]:
find_matching_face('/content/AI-Face-Identifier/face test image/download1.png', em1) # younger maybe

{'match_found': True, 'similarity': np.float64(0.6890078163488114)}

In [49]:
find_matching_face('/content/AI-Face-Identifier/face test image/images3.png', em1) # diffrent style

{'match_found': True, 'similarity': np.float64(0.5594492864025551)}

In [50]:
find_matching_face('/content/AI-Face-Identifier/face test image/download4.png', em1) # trump

{'match_found': False, 'similarity': np.float64(0.02054582041886698)}

In [51]:
find_matching_face('/content/AI-Face-Identifier/face test image/download3.png', em1) # angelina joli

{'match_found': False, 'similarity': np.float64(0.1120935080123743)}